# Modelo A para clasificación de imagenes

#### Jeffrey Daniel Leiva Cascante 2021016720
#### Richard Osvaldo León Chinchilla 2019003759

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.4.1+cu118
True
